In [2]:
import tensorflow as tf
import numpy as np
import h5py
import matplotlib.pyplot as plt
from importlib import reload

In [2]:
import models.models as models
reload(models)
import utils.preprocessing_L1 as prepr
reload(prepr)

<module 'utils.preprocessing' from '/eos/home-n/nchernya/MLHEP/AnomalyDetection/ADgvae/utils/preprocessing.py'>

In [3]:
#BG
data_dir = 'datasets/L1_ADC_2021/'
filename_bg =  data_dir + 'background_for_training.h5'
batch_size = 128
nodes_n, feat_sz, particles_bg, A_bg, A_tilde_bg = prepr.prepare_data(filename_bg,0,batch_size*20)

/eos/home-n/nchernya/MLHEP/AnomalyDetection/ADgvae/utils/preprocessing.py:20: RuntimeWarning: divide by zero encountered in power
  D = np.nan_to_num(np.power(D,-0.5), posinf=0, neginf=0) # normalize (**-(1/2))


In [4]:
print(particles_bg.shape, A_tilde_bg.shape, A_bg.shape)

(2560, 19, 4) (2560, 19, 19) (2560, 19, 19)


# Autoencoder

In [5]:
gnn = models.GraphAutoencoder(nodes_n=nodes_n, feat_sz=feat_sz, activation=tf.nn.tanh)
gnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), run_eagerly=True)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input_features (InputLa [(None, 19, 4)]      0                                            
__________________________________________________________________________________________________
encoder_input_adjacency (InputL [(None, 19, 19)]     0                                            
__________________________________________________________________________________________________
graph_convolution (GraphConvolu (None, 19, 6)        54          encoder_input_features[0][0]     
                                                                 encoder_input_adjacency[0][0]    
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 19, 8)        104         graph_convolution[0][0

In [6]:
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5, verbose=1)]
gnn.fit((particles_bg, A_tilde_bg), A_bg, epochs=100, batch_size=128, validation_split=0.25, callbacks=callbacks)

Epoch 1/100
15/15 [==============================] - 1s 48ms/step - val_loss: 0.7328
Epoch 2/100
15/15 [==============================] - 1s 34ms/step - val_loss: 0.6731
Epoch 3/100
15/15 [==============================] - 1s 34ms/step - val_loss: 0.6582
Epoch 4/100
15/15 [==============================] - 1s 36ms/step - val_loss: 0.6527
Epoch 5/100
15/15 [==============================] - 0s 33ms/step - val_loss: 0.6475
Epoch 6/100
15/15 [==============================] - 1s 35ms/step - val_loss: 0.6423
Epoch 7/100
15/15 [==============================] - 0s 33ms/step - val_loss: 0.6371
Epoch 8/100
15/15 [==============================] - 0s 30ms/step - val_loss: 0.6321
Epoch 9/100
15/15 [==============================] - 1s 37ms/step - val_loss: 0.6274
Epoch 10/100
15/15 [==============================] - 0s 33ms/step - val_loss: 0.6230
Epoch 11/100
15/15 [==============================] - 1s 33ms/step - val_loss: 0.6190
Epoch 12/100
15/15 [==============================] - 0s 32ms/s

15/15 [==============================] - 0s 32ms/step - val_loss: 0.5522
Epoch 97/100
15/15 [==============================] - 0s 32ms/step - val_loss: 0.5521
Epoch 98/100
15/15 [==============================] - 1s 35ms/step - val_loss: 0.5520
Epoch 99/100
15/15 [==============================] - 1s 37ms/step - val_loss: 0.5517
Epoch 100/100
15/15 [==============================] - 1s 34ms/step - val_loss: 0.5515


## Predict BG and Sig

In [7]:
#signal
#filename_sig = 'Ato4l_lepFilter_13TeV.h5'
#filename_sig='leptoquark_LOWMASS_lepFilter_13TeV.h5'
filename_sig = data_dir + 'hToTauTau_13TeV_PU20.h5'
_,_, particles_sig, A_sig, A_tilde_sig = prepr.prepare_data(filename_sig)

#background test 
_,_, particles_bg_test, A_bg_test, A_tilde_bg_test = prepr.prepare_data(filename_bg,3000,4000)


In [1]:
z_sig, A_pred_sig = gnn((particles_sig, A_tilde_sig))
z_bg_test, A_pred_bg_test = gnn((particles_bg_test, A_tilde_bg_test))
loss_signal = tf.math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(A_sig, A_pred_sig), axis=(1,2)).numpy()
loss_bg = tf.math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(A_bg_test, A_pred_bg_test), axis=(1,2)).numpy()

NameError: name 'gnn' is not defined

In [ ]:
max_score = np.max(np.max(loss_bg))#,np.max(loss_bg_dist))
# plot BG
plt.figure()
plt.hist(loss_bg, bins=100, label='BG', density=True, range=(0, max_score), 
         histtype='step', fill=False, linewidth=1.5,color='g')
plt.hist(loss_signal, bins=100, label='SIG', density=True, range=(0, max_score), 
         histtype='step', fill=False, linewidth=1.5,color='r')
plt.semilogy()
plt.xlabel("AE Loss")
plt.ylabel("Probability (a.u.)")
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

# Trying GCN VAE

In [ ]:
def threeD_loss_manual(inputs, outputs):
    distances = np.sum(np.subtract(inputs[:,:,np.newaxis,:],outputs[:,np.newaxis,:,:])**2, axis=-1)
    min_dist_to_inputs = np.min(distances,axis=1)
    min_dist_to_outputs = np.min(distances,axis=2)
    return np.sum(min_dist_to_inputs,axis=1) + np.sum(min_dist_to_outputs,axis=1)



In [ ]:
from importlib import reload
import model
reload(model)
from model import *
gcnvae = GCNVariationalAutoEncoder(nodes_n=nodes_n, feat_sz=feat_sz, activation=tf.nn.tanh)
gcnvae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), run_eagerly=True)

In [ ]:
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5, verbose=1)]
gcnvae.fit(particles_bg, A_bg, epochs=100, batch_size=128,callbacks=callbacks) # validation_split=0.25, 

In [ ]:
#background

In [ ]:
_,_, particles_bg_test, A_bg_test, A_tilde_bg_test = prepare_data(filename_bg,3000,4000)
features_out_bg_test, z_bg_test, z_mean_bg_test, z_log_var_bg_test = gcnvae((particles_bg_test, A_bg_test))
features_out_bg_test = features_out_bg_test.numpy().astype('float32')
loss_bg = threeD_loss(particles_bg_test.astype('float32'),features_out_bg_test).numpy()

In [ ]:
#signal
#filename_sig = 'Ato4l_lepFilter_13TeV.h5'
#filename_sig='leptoquark_LOWMASS_lepFilter_13TeV.h5'
filename_sig = 'hToTauTau_13TeV_PU20.h5'
nodes_n, feat_sz, particles_sig, A_sig, A_tilde_sig = prepare_data(filename_sig)

features_out_sig, z_sig, z_mean_sig, z_log_var_sig = gcnvae((particles_sig, A_sig))
features_out_sig = features_out_sig.numpy().astype('float32')
loss_signal = threeD_loss(particles_sig.astype('float32'),features_out_sig).numpy()

In [ ]:
max_score = np.max(np.max(loss_bg))#,np.max(loss_bg_dist))
# plot BG
plt.figure()
plt.hist(loss_bg, bins=100, label='BG', density=True, range=(0, max_score), 
         histtype='step', fill=False, linewidth=1.5,color='g')
plt.hist(loss_signal, bins=100, label='SIG', density=True, range=(0, max_score), 
         histtype='step', fill=False, linewidth=1.5,color='r')
#plt.semilogy()
plt.xlabel("AE Loss")
plt.ylabel("Probability (a.u.)")
plt.grid(True)
plt.legend(loc='upper right')
plt.show()

In [ ]:
loss_signal.mean()

In [ ]:
loss_bg.mean()